In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


class Hospital:
    def __init__(self, name, occupancy, available_equipment, distance):
        self.name = name
        self.occupancy = occupancy
        self.available_equipment = available_equipment
        self.distance = distance

class Patient:
    def __init__(self, symptoms, required_equipment=None):
        self.symptoms = symptoms
        self.required_equipment = required_equipment
    
    def predict_required_equipment(self):
        symptom_equipment_mapping = {
            "fever": ["oxygen"],
            "cough": ["oxygen"],
            "shortness_of_breath": ["ventilator"],
            "fatigue": ["oxygen"]
#             "heart": ["defibrillator"],
            # Add more mappings based on you1r knowledge of symptoms and equipment requirements
        }
        
        predicted_equipment = set()
        for symptom in self.symptoms:
            if symptom in symptom_equipment_mapping:
                predicted_equipment.update(symptom_equipment_mapping[symptom])
        
        self.required_equipment = list(predicted_equipment)
        return self.required_equipment


# Severity assessment model
class SeverityAssessmentModel:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    def preprocess_input(self, symptoms):
        # Preprocessing steps specific to your data
        symptom_mapping = {"fever": 1, "cough": 2, "shortness_of_breath": 3, "fatigue": 4,"heart":5}
        symptom_vector = [symptom_mapping[symptom] for symptom in symptoms]
        return np.array(symptom_vector).reshape(1, -1)
    
    def train_model(self, X, y):
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)
    
    def predict_severity(self, input_data):
        input_features = self.preprocess_input(input_data)
        input_features_scaled = self.scaler.transform(input_features)
        severity_prediction = self.model.predict(input_features_scaled)
        return severity_prediction


def evaluate_hospitals(patient, hospitals, max_occupancy):
    suitable_hospitals = []
    
    for hospital in hospitals:
        if (hospital.occupancy < max_occupancy and
            all(equipment in hospital.available_equipment for equipment in patient.required_equipment)):
            suitable_hospitals.append(hospital)
            
    suitable_hospitals.sort(key=lambda x: x.distance)
    return suitable_hospitals

def allocate_patient(patient, hospitals, max_occupancy,n):
    severity_model = SeverityAssessmentModel()
    # Load and train the model with your actual training data (X_train, y_train)
    X_train = np.random.rand(10,n)  # Update this to match the actual number of features in your training data
    y_train = np.random.choice(["mild", "moderate", "severe"], size=10)  # Replace with actual severity labels
    severity_model.train_model(X_train, y_train)
    patient.predict_required_equipment()  # Predict required equipment based on symptoms
    
    severity_level = severity_model.predict_severity(patient.symptoms)
    suitable_hospitals = evaluate_hospitals(patient, hospitals, max_occupancy)
    
    # Implement your allocation strategy (e.g., assign patient to the first suitable hospital)
    if suitable_hospitals:
        allocated_hospital = suitable_hospitals[0]
        print("Allocating patient to:", allocated_hospital.name)
        print("Required Equipment:", patient.required_equipment)
    else:
        print("No suitable hospital available.")
        
hospital_data = [
    Hospital(name="Hospital A", occupancy=0.7, available_equipment=["oxygen","ventilator"], distance=5),
    Hospital(name="Hospital B", occupancy=0.9, available_equipment=["ventilator"], distance=5),
    Hospital(name="Hospital C", occupancy=0.1, available_equipment=["oxygen"], distance=6),
    Hospital(name="Hospital D", occupancy=0.4, available_equipment=["oxygen"], distance=8),
    Hospital(name="Hospital E", occupancy=0.5, available_equipment=["ventilator"], distance=2)
]

patient_symptoms = ["cough","shortness_of_breath","fever"]
patient = Patient(symptoms=patient_symptoms)

# Allocate the patient to a suitable hospital
MAX_OCCUPANCY = 0.9  # Example maximum occupancy threshold
allocate_patient(patient, hospital_data, MAX_OCCUPANCY,len(patient_symptoms))

Allocating patient to: Hospital A
Required Equipment: ['oxygen', 'ventilator']


In [2]:
import heapq
import random

class Sample:
    def __init__(self, id, urgency):
        self.id = id
        self.urgency = urgency

    # Define comparison methods for Sample instances
    def __lt__(self, other):
        return self.urgency < other.urgency
    
    def __eq__(self, other):
        return self.urgency == other.urgency

# Laboratory Testing Optimization using Priority Queue
def optimize_lab_testing(samples, lab_capacity):
    priority_queue = samples.copy()
    heapq.heapify(priority_queue)

    allocated_samples = []
    while lab_capacity > 0 and priority_queue:
        sample = heapq.heappop(priority_queue)
        allocated_samples.append(sample)
        lab_capacity -= 1

    return allocated_samples

if __name__ == "__main__":
    num_samples = 20
    lab_capacity = 5

    samples = [Sample(i, random.randint(1, 10)) for i in range(num_samples)]
    allocated_samples = optimize_lab_testing(samples, lab_capacity)

    print("Allocated Samples:")
    for sample in allocated_samples:
        print(f"Sample {sample.id}, Urgency: {sample.urgency}")

Allocated Samples:
Sample 12, Urgency: 1
Sample 17, Urgency: 3
Sample 8, Urgency: 3
Sample 14, Urgency: 4
Sample 10, Urgency: 4
